# Case Study: OpenStreetMap Data
### Author:  Daniel J. Lee
### Date: September 11, 2017

## 1. Purpose

The purpose of this case study is to examine the process of auditing the OpenStreetMap XML data for <b>Atlanta, GA</b>. The process involved examining the overall structure and cleaning issues in our data. The XML data was later converted into CSV format and submitted to <b>sqlite3</b> for further analysis.

## 2. Format
<br>
<p> <img src="https://i.imgur.com/aXRID8j.png" align="left" style="width: 40%; height: 40%"></p>
<br>

<li type="square"><b>atlanta_georgia.osm - [289.9 MB]</b> - The OSM extract of Atlanta, GA that was obtained from [Mapzen](https://mapzen.com/data/metro-extracts/metro/atlanta_georgia/). I chose Atlanta, GA specifically because I wanted to know more about the city I am from.</li>
<li type="square"><b>sample.osm - [73.3 MB]</b> - Due to issues of running the original OSM extract on the local computer, we decided to use a portion of the OSM data using <i>create_sample.py</i></li>
<li type="square"><b>analysis.py</b> - Included to count the number of various elements in our OSM file.</li>
<li type="square"><b>audit.py</b> - Various functions to clean our data with format issues.</li>
<li type="square"><b>schema.py</b> - Used to validate our structure when importing to CSV</li>
<li type="square"><b>main.py</b> - Processes OSM, cleans, and convert to CSV</li>

The following is an sample of the OSM XML file:

In [ ]:
<way changeset="438149" id="6254528" timestamp="2007-09-16T02:16:49Z" uid="7168" user="DaveHansenTiger" version="1">
	<nd ref="52390437" />
	<nd ref="52390439" />
	<nd ref="52390441" />
	<nd ref="52390443" />
	<nd ref="52390444" />
	<nd ref="52390446" />
	<tag k="highway" v="residential" />
	<tag k="tiger:cfcc" v="A41" />
	<tag k="tiger:tlid" v="3061719" />
	<tag k="tiger:county" v="Chambers, AL" />
	<tag k="tiger:source" v="tiger_import_dch_v0.6_20070808" />
	<tag k="tiger:reviewed" v="no" />
	<tag k="tiger:separated" v="no" />
	<tag k="tiger:upload_uuid" v="bulk_upload.pl-c08d5681-dfbe-4263-a58f-d8cbd01bee56" />
</way>

<li type="square">We want to convert the file to the format (e.g way, way_nodes, way_tags): </li>

In [ ]:
{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}

## 3. Analysis

A better understanding of the data was required before cleaning and writing the file to CSV. The following functions are stored in <b>analysis.py</b> were used to kept track of the occurance using dictionaries for each element and their respective tags.

### 3a. Definition

In [ ]:
count_tag(tree)

{'member': 966,
 'nd': 348328,
 'node': 305187,
 'osm': 1,
 'relation': 167,
 'tag': 153954,
 'way': 37391}

According to OpenStreetMap Wiki, the basic components of OpenStreetMap's conceptual data model of the physical world are:



<li type="square"><b>nodes</b> - define points in space</li>
<li type="square"><b>ways</b> - define linear features and area boundaries</li>
<li type="square"><b>relations</b> -  explain how other elements work together</li>


Furthermore, <b>tags</b> are used to define a specific key and value of a element. 

In [ ]:
<tag k="name" v="Budd Terrace - Wesley Woods Intermediate Care Facility" />
<tag k="amenity" v="hospital" />

<li type="square"><b>nd</b> and <b>member</b> are used to convey further information to describe <b>way</b> and <b>member</b> respectively.</li>

### 3b. Structure

In order to examine the structure, <b>count_attribute</b> was used to determine the attributes for each element.

In [ ]:
count_attribute(tree)

node
{'changeset': 305187,
 'id': 305187,
 'lat': 305187,
 'lon': 305187,
 'timestamp': 305187,
 'uid': 305187,
 'user': 305187,
 'version': 305187}
node/tag
{'k': 13246, 'v': 13246}
way
{'changeset': 37391,
 'id': 37391,
 'timestamp': 37391,
 'uid': 37391,
 'user': 37391,
 'version': 37391}
way/nd
{'ref': 348328}
way/tag
{'k': 139893, 'v': 139893}

The following function <b>unique_key</b> was created in <b>analysis.py</b> to determine the occurances of keys from our tags. I decided to show the 20 highest occurances. There were 397 total unique keys in our sample OSM file.

In [ ]:
unique_key(tree)

[('building', 31783),
 ('addr:housenumber', 19967),
 ('addr:street', 19962),
 ('addr:city', 19894),
 ('addr:postcode', 19644),
 ('highway', 4437),
 ('name', 3012),
 ('DeKalb:id', 2120),
 ('tiger:county', 2067),
 ('tiger:cfcc', 2067),
 ('source', 1883),
 ('tiger:name_base', 1809),
 ('tiger:reviewed', 1773),
 ('tiger:name_type', 1604),
 ('tiger:zip_left', 1462),
 ('tiger:name_direction_suffix', 1457),
 ('tiger:zip_right', 1428),
 ('amenity', 1117),
 ('oneway', 869),
 ('service', 616)]

From <b>analysis.py</b>, I examined the keys using function <b>explore_path</b> to get an understanding of how the values were listed. I decided to focus on the following keys that deemed to have issues. 
<br>
<br>
<li type="square"> addr:street </li>
<li type="square"> phone </li>
<li type="square"> addr:state </li>
<li type="square"> addr:postcode </li>
<li type="square"> tiger:county </li>
<li type="square"> addr:county </li>

## 4. Problem Detection



### 4a. Street Names (addr:street)

By carefully examining the dataset, we can see the inconsistencies in the street suffixes - some as abbreviations whereas some not.

From <b>audit.py</b>, the function <b>update_street</b> determines whether the passed suffix is registered in variable <b>expected</b> which is our list of suffixes. The variable <b>mapping</b> is used to address and correct abbreviations. Furthermore, a dictionary <b>street_issue</b> was created to keep track of unaddressed issues. <b>street_issue</b> keeps count of street suffixes and examined to determine whether it is an actual suffix or not.

In [ ]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Lane", "Road", 
            "Trail", "Parkway", "Ridge", "Way", "Pass", "Look", "Creek", "Chase", "Crossing",
            "Terrace", "Point", "Path", "Run", "Cove", 'Bend', 'Circle', 'Trace', 'Walk',
            "Southeast", "Southwest", "Northeast", "Northwest", "View", "Landing", "North","East",
            "South", "West"]

In [ ]:
mapping = { "St": "Street",
                "St.": "Street",
                "Blvd": "Boulevard",
                "Blvd.": "Boulevard",
                "Ave": "Avenue",
                "Ave.": "Avenue",
                "Rd.": "Road",
                "Rd" : "Road,",
                "Dr" : "Drive",
                ...
                "Pt": "Point",
                "S": "South",
                "S.": "South",
                "W": "West",
                "W.": "West",
                "N": "North",
                "N.": "North",
                "E": "East",
                "E.": "East",
                "NE": "Northeast",
                "NW": "Northwest",
                "SE": "Southeast",
                "SW": "Southwest",
                "Hts": "Heights",
                "Rte": "Route"}

#### Several points came about:

<li type="square">Cardinals had to be addressed.</li>


In [ ]:
Roswell Road NE ==> Roswell Road Northeast

<li type="square">Capitalization was a minor issue.</li>

In [ ]:
Riverwood SPG ==> Riverwood Spg
Waters RUN ==> Waters Run
Creel lane ==> Creel Lane

<li type="square">There are some street names that do not contain suffixes</li>

In [ ]:
{'Addison Park': 'Park',
 'Barrow Downs': 'Downs',
 'Creekwood': 'Creekwood',
 'Diamond Bluff': 'Bluff',
 'Dunvegan Close': 'Close',
 'Farm Track': 'Track',
 'Gable Gate Turn': 'Turn',
 'Hope Vine': 'Vine'}

#### Conclusion:
Overall, we wanted to make sure that the suffix was not abbreviated. We also did not want to flag any suffix that was appropriate (e.g. Avenue). 


### 4b. Phone Numbers (phone)

Phone numbers did not have a standard format. '1+' was often used and spacing was considered an issue. Thus the format: {3}-{3}-{4} was used to keep consistency.

In [ ]:
Before:
+1-404-754-9290
+1-404-883-3406
(706) 372-2455
678-691-5238
+1 (770) 612-9423
+1 (770) 612-9423
1-678-366-3360
    
After:
404-754-9290
404-883-3406
706-372-2455
678-691-5238
770-612-9423
770-612-9423
678-366-3360

### 4c. State Codes (addr:state)

State code displayed inconsistent format. In order to keep an appropriate format, <i>GA</i> was used.


In [ ]:
Before:
[('GA', 216), ('ga', 5), ('Ga', 1)]

After:
[('GA', 222)]

### 4d. Postal codes (addr:postcode)

US postal codes have a basic 5-digit format but several postal codes encountered had an additional four digits. It was introduced by the U.S Postal Services to have more accurate information within certain postal codes. I decided to keep the 5-digit format for all postal codes for this case study.

In [ ]:
Before:
30058
30083
30083-1218
30083-1233
30083
    
After:
30058
30083
30083
30083
30083

### 4e. Counties (tiger:county, addr:county)

There are entities that contained multiple counties that were separated by either ":" or ";" as shown below. It is interesting to note that the first county specified also was <i>AL</i>. If the first county had a<i>AL</i> state code, we specified the county as "Fulton, GA" which is the county that Atlanta, GA is located.

In [ ]:
 ('Calhoun, AL:Carroll, GA:Cleburne, AL:Cobb, GA:Douglas, GA:Fulton, GA:Haralson, GA:St. Clair, AL:Talladega, AL',
  4),

A dictonary was created to verify the exact amount when converting to CSV. The following below displays the 8 counties with the highest occurance. 

In [ ]:
[('Cherokee, GA', 1073),
 ('Paulding, GA', 475),
 ('Cobb, GA', 351),
 ('Fulton, GA', 346),
 ('Gwinnett, GA', 249),
 ('DeKalb, GA', 228),
 ('Rockdale, GA', 193),
 ('Carroll, GA', 89)]

# 5. SQLite3

After cleaning the data, we converted the file to CSV and ready to submit to SQLite3. The following was performed in our local terminal and database:

### 5a. Structure

In [2]:
import sqlite3
conn = sqlite3.connect("p3.db")
cursor = conn.cursor()

In [3]:
query = "SELECT COUNT(*) FROM nodes;"
cursor.execute(query)
print 'There are {} nodes in our database. \n'.format(cursor.fetchall()[0][0])

query = "SELECT COUNT(*) FROM ways;"
cursor.execute(query)
print 'There are {} ways in our database.'.format(cursor.fetchall()[0][0])

There are 305187 nodes in our database. 

There are 37391 ways in our database.


<li type="square">We were able to verify that there were no loss of nodes / ways when we converted the file from OSM to CSV.</li>

### 5b. Top 5 Users

In [4]:
query = """
SELECT user, COUNT(*)
FROM nodes
GROUP BY user
ORDER BY COUNT(*) DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[(u'Saikrishna_FultonCountyImport', 230013),
 (u'Jack Kittle Buildings', 19312),
 (u'woodpeck_fixbot', 10808),
 (u'Liber', 10317),
 (u'Ryan Lash', 5825)]

### 5c. Unique Users

In [17]:
query="""
SELECT COUNT(DISTINCT(sub.uid))          
FROM (SELECT uid FROM nodes UNION SELECT uid FROM ways) as sub;
"""
cursor.execute(query)
print 'There are {} unique users in our database.'.format(cursor.fetchall()[0][0])

There are 536 unique users in our database.


### 5c. Top 5 Cuisines

In [4]:
query = """
SELECT nodes_tags.value, COUNT(*)
FROM nodes_tags
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') as sub ON nodes_tags.id = sub.id
WHERE nodes_tags.key = 'cuisine'
GROUP BY nodes_tags.value
ORDER BY COUNT(*) DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[(u'american', 17),
 (u'mexican', 5),
 (u'pizza', 5),
 (u'chinese', 4),
 (u'italian', 4)]

### 5d. Top 5 Amenities

In [24]:
query = """
SELECT nodes_tags.value, COUNT(*)
FROM nodes_tags
WHERE nodes_tags.key = 'amenity'
GROUP BY nodes_tags.value
ORDER BY COUNT(*) DESC
LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[(u'restaurant', 78),
 (u'place_of_worship', 54),
 (u'school', 38),
 (u'bicycle_parking', 28),
 (u'fast_food', 23)]

### 5e. Name of Fast Food Restaurants

In [27]:
query = """
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='fast_food') as sub ON nodes_tags.id = sub.id
WHERE nodes_tags.key = 'name'
GROUP BY value
ORDER BY num DESC;
"""
cursor.execute(query)
cursor.fetchall()

[(u'Subway', 4),
 (u"McDonald's", 3),
 (u"Arby's", 1),
 (u'Burger King', 1),
 (u'Checkers Drive-In', 1),
 (u'Chick-Fil-A', 1),
 (u'China Dragon', 1),
 (u"Church's", 1),
 (u'Cleveland Style Deli', 1),
 (u"Domino's Pizza", 1),
 (u'Five Guys', 1),
 (u'Flats', 1),
 (u"Jimmy John's", 1),
 (u'Panera Bread', 1),
 (u"Papa John's", 1),
 (u'Pizza Hut', 1),
 (u"Rosa's Pizza", 1)]

# 6. Conclusion

Throughout this project, we were able to successfully convert an OpenStreetMap file to CSV and submit to Sqlite3 for further analysis. During the conversion process, we saw various issues such as street suffixes and inconsistent phone formats. We were able to verify that there were no loss of file during that conversion as well by examining the structure in SQLite. By using queries, we were able to obtain interesting information from our dataset as well.

## 6a. Suggestions: name

After performing queries on SQLite3, one question that came about was whether <i>names</i> had issues as well. I examined fast food restaurant names in the section above and there seemed to be no issues. However I created another database with the original file <b>atlanta_ga.osm</b> with the same format for elements. I ran the query for name of fast food restaurants (5e) again and noticed some issues among names:

In [ ]:
[(u'Subway', 11),
 (u"McDonald's", 7),
 (u"Jimmy John's", 4),
 (u"Wendy's", 4),
 (u"Arby's", 3),
 (u'Burger King', 3),
 (u'Taco Bell', 3),
 (u'Chick-fil-A', 2),
 (u'Five Guys', 2),
 (u"Jersey Mike's Subs", 2),
 (u'Pizza Hut', 2),
 (u"Alibaba's Turkish Food", 1),
 (u'Atlantic Cafe', 1),
 (u'Burger Win', 1),
 (u'Checkers Drive-In', 1),
 (u'Chick-Fil-A', 1),
 (u'China Dragon', 1),
 (u'Chipotle Mexican Grill', 1),
 (u"Church's", 1),
 (u'Cleveland Style Deli', 1),
 (u'Cookout', 1),
 (u'Dairy Queen', 1),
 (u"Domino's", 1),
 (u"Domino's Pizza", 1),
 (u'Dunkin Donuts', 1),
 (u'Einstein Bros Bagels', 1),
 (u'Flats', 1),
 (u'Green Apple Pizza', 1),
 (u'Happy Donuts', 1),
 (u"Jason's Deli", 1),
 (u"Krystal's", 1),
 (u"Long John Silver's", 1),
 (u"Moe's", 1),
 (u"Moe's Burritos", 1),
 (u'Naan Stop', 1),
 (u'Panera Bread', 1),
 (u"Paolo's Gelato Italiano", 1),
 (u"Papa John's", 1),
 (u'Planet Smoothie', 1),
 (u"Rosa's Pizza", 1),
 (u'Sensational Subs', 1),
 (u"Shane's Rib Shack", 1),
 (u'Smash burger', 1),
 (u'Smoothie King', 1),
 (u'We Suki Suki', 1),
 (u'WhichWhich', 1),
 (u"Willy's", 1),
 (u"Willy's Mexican Grill", 1),
 (u'Willys Mexican', 1),
 (u'Wingnutz', 1),
 (u'Zesto', 1),
 (u"Zoe's Kitchen ", 1)]

<li type="square">We can see that <b>Willy's Mexican Grill</b> had two other variations: <b>Willy's</b> and <b>Willys Mexican</b>. Thus one of the suggestions to improving the OSM data is to also determining a method in addressing names of restauraunts (and other areas) as well. The benefits could allow users to accurately determine how many restauraunts are in a particular area. For example, if we were interested in determining how many [Moe's](https://www.moes.com) are in our area, we would expect the result to be <b>2</b>. An anticipated problem in implementing this method includes as how to group these names (e.g <b>Moe's</b> or <b>Moe's Burritos</b>).

# 7. Reference

https://docs.python.org/2/library/xml.etree.elementtree.html<br>
https://en.wikipedia.org/wiki/ZIP_Code<br>
http://www.worldatlas.com/na/us/al/c-randolph-county-alabama.html<br>
https://www.tutorialspoint.com/python/list_list.htm<br>
https://wiki.openstreetmap.org/wiki/Elements<br>
https://stackoverflow.com/questions/11390556/processing-xml-in-python-with-elementtree<br>
https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md<br>